In [2]:
import pandas as pd
#import MetaTrader5 as mt5
import talib as ta
from bb_setup_1 import setup_2,main_indicators
import warnings
warnings.filterwarnings("ignore")

In [ ]:
ativo = 'wdo_1_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
df = df[df['Data'] > '2022-06-01']

In [17]:
pontos = 3
rate_stop = 1
rate_tp = 2
spread = 5

df_total = None

for timeframe in [5]:
    for var_bb in [2]:
        for pontos in [25,30,40]:
            for rate_tp in [1.5,2,2.5,3]:
                
                ativo = f'win_{timeframe}_full'
                colnames = ['Data','Open','High','Low','Close','Vol']
                df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
                df = df[df['Data'] > '2022-06-01']

                df_acao = setup_2(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)

                qtd_operacoes = df_acao.shape[0]
                taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
                operacoes_certas = qtd_operacoes * taxa_acerto
                operacoes_erradas = qtd_operacoes - operacoes_certas
                pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

                pontos_liquido = pontos_bruto - spread * qtd_operacoes


                df_new = pd.DataFrame([[ativo[:-5],var_bb,rate_tp,rate_stop,pontos,qtd_operacoes,taxa_acerto,pontos_bruto,pontos_liquido]],
                            columns = ['ativo','var_bb','rate_tp','rate_stop','pontos','qtd_operacoes','taxa_acerto','pontos_bruto','pontos_liquido'])

                df_total = pd.concat([df_total,df_new])

                print(df_total.to_string())

   ativo  var_bb  rate_tp  rate_stop  pontos  qtd_operacoes  taxa_acerto  pontos_bruto  pontos_liquido
0  win_5       2      1.5          1      25            613     0.769984       14175.0         11110.0
   ativo  var_bb  rate_tp  rate_stop  pontos  qtd_operacoes  taxa_acerto  pontos_bruto  pontos_liquido
0  win_5       2      1.5          1      25            613     0.769984       14175.0         11110.0
0  win_5       2      2.0          1      25            613     0.707993       17225.0         14160.0
   ativo  var_bb  rate_tp  rate_stop  pontos  qtd_operacoes  taxa_acerto  pontos_bruto  pontos_liquido
0  win_5       2      1.5          1      25            613     0.769984       14175.0         11110.0
0  win_5       2      2.0          1      25            613     0.707993       17225.0         14160.0
0  win_5       2      2.5          1      25            613     0.606852       17225.0         14160.0
   ativo  var_bb  rate_tp  rate_stop  pontos  qtd_operacoes  taxa_acerto 

In [5]:
df_total = pd.read_csv('result_bb_setup_2_win.csv')
#df_total.to_csv('result_bb_setup_2_wdo.csv')

In [18]:
#df_1 = df_total[df_total['ativo'] == 'win_1']
df_5 = df_total[df_total['ativo'] == 'win_5']
df_15 = df_total[df_total['ativo'] == 'win_15']

In [19]:
df_5.sort_values(by = 'pontos_liquido', ascending = False).head(10)

,ativo,var_bb,rate_tp,rate_stop,pontos,qtd_operacoes,taxa_acerto,pontos_bruto,pontos_liquido
0,win_5,2,3.0,1,30,613,0.517129,19650.0,16585.0
0,win_5,2,3.0,1,25,613,0.566069,19375.0,16310.0
0,win_5,2,2.5,1,30,613,0.567700,18150.0,15085.0
0,win_5,2,2.5,1,40,613,0.489396,17480.0,14415.0
0,win_5,2,2.0,1,25,613,0.707993,17225.0,14160.0
0,win_5,2,2.5,1,25,613,0.606852,17225.0,14160.0
0,win_5,2,2.0,1,30,613,0.637847,16800.0,13735.0
0,win_5,2,3.0,1,40,613,0.420881,16760.0,13695.0
0,win_5,2,2.0,1,40,613,0.559543,16640.0,13575.0
0,win_5,2,1.5,1,30,613,0.734095,15360.0,12295.0


In [15]:
df_15.sort_values(by = 'pontos_liquido', ascending = False).head(20)

,ativo,var_bb,rate_tp,rate_stop,pontos,qtd_operacoes,taxa_acerto,pontos_bruto,pontos_liquido
0,win_15,2,3.0,1,40,211,0.654028,13640.0,12585.0
0,win_15,2,3.0,1,30,211,0.720379,11910.0,10855.0
0,win_15,2,2.5,1,40,211,0.677725,11580.0,10525.0
0,win_15,2,3.0,1,25,211,0.763033,10825.0,9770.0
0,win_15,2,2.0,1,40,211,0.753555,10640.0,9585.0
0,win_15,2,2.5,1,30,211,0.763033,10575.0,9520.0
0,win_15,2,2.5,1,25,211,0.781991,9162.5,8107.5
0,win_15,2,2.0,1,30,211,0.796209,8790.0,7735.0
0,win_15,2,1.5,1,40,211,0.796209,8360.0,7305.0
0,win_15,2,2.0,1,25,211,0.829384,7850.0,6795.0


In [11]:
timeframe = 15
for index,row in df_15.sort_values(by = 'pontos_liquido', ascending = False).loc[[17]].iterrows():

    pontos = row['pontos']
    rate_tp = row['rate_tp']
    var_bb = row['var_bb']
    rate_stop = 1
    spread = 0.5

    ativo = f'win_{timeframe}_full'
    colnames = ['Data','Open','High','Low','Close','Vol']
    df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
    df = df[df['Data'] > '2022-01-01']

    df_acao = setup_2(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)

    qtd_operacoes = df_acao.shape[0]
    taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes


    print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

    print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

    df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
    df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
    df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

    df_new_mes = df_acao[['Mes','resultado_binario']]
    df_new_hora = df_acao[['Hora_h','resultado_binario']]

    def calc_pontos(x):
        qtd_operacoes = x['qtd_operacoes']
        taxa_acerto = x['taxa_acerto']
        operacoes_certas = qtd_operacoes * taxa_acerto
        operacoes_erradas = qtd_operacoes - operacoes_certas
        pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

        pontos_liquido = pontos_bruto - spread * qtd_operacoes

        return pontos_liquido

    print('Agrupado por Hora')
    df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

    df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
    print(df_new_hora)

    print('Agrupado por Mes')
    df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

    df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
    print(df_new_mes)
    print('')


Timeframe: 15 --pontos: 25 --- rate_tp: 2.0 -- var_bb: 2
Pontos_liquido: 14161.5 -- Total_operacoes: 377
Agrupado por Hora
  Hora_h  qtd_operacoes  taxa_acerto  pontos_liquido
0     09             83     0.891566          3433.5
1     10             51     0.901961          2149.5
2     11             35     0.857143          1357.5
3     12             22     1.000000          1089.0
4     13             10     0.900000           420.0
5     14             31     0.838710          1159.5
6     15             42     0.785714          1404.0
7     16             52     0.788462          1749.0
8     17             43     0.697674          1153.5
9     18              8     0.750000           246.0
Agrupado por Mes
   Mes  qtd_operacoes  taxa_acerto  pontos_liquido
0   01             36     0.888889          1482.0
1   02             30     0.800000          1035.0
2   03             33     0.727273           958.5
3   04             32     0.968750          1509.0
4   05             35 

### Modelo

In [1]:
import pandas as pd
#import MetaTrader5 as mt5
import talib as ta
from bb_setup_1 import setup_2,main_indicators
import warnings
warnings.filterwarnings("ignore")

In [2]:
timeframe = 15
pontos = 30
rate_tp = 2
var_bb = 2

ativo = f'win_{timeframe}_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
df = df[df['Data'] > '2022-06-01']

df_acao = setup_2(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)
df_acao['dist_bandas'] = df_acao['BB_up'] - df_acao['BB_low']

df_acao = main_indicators(df_acao)

In [26]:
df_acao[['Data','acao', 'stop','tp','resultado_binario']].tail(10)

,Data,acao,stop,tp,resultado_binario
45098,2022-12-14 15:00:00,call,105215.0,105335.0,1.0
45118,2022-12-15 10:30:00,call,106570.0,106690.0,0.0
45150,2022-12-16 09:00:00,sell,105175.0,105055.0,1.0
45163,2022-12-16 12:15:00,sell,104475.0,104355.0,1.0
45184,2022-12-16 17:30:00,call,104845.0,104965.0,1.0
45200,2022-12-19 12:00:00,call,105790.0,105910.0,1.0
45231,2022-12-20 10:15:00,call,107665.0,107785.0,1.0
45263,2022-12-20 18:15:00,call,109290.0,109410.0,1.0
45305,2022-12-22 09:45:00,call,109510.0,109630.0,1.0
45325,2022-12-22 14:45:00,sell,108825.0,108705.0,1.0


In [3]:
df_acao['resultado_binario'].sum()/df_acao.shape[0]

0.7962085308056872

In [329]:
df = df_acao.copy()

df = df.drop(columns = ['proximo_candle','stop','tp','TRIX'])
df = df.dropna()
inicio = '2022-11-01'
termino = '2022-01-01'

df_train = df[(df['Data'] > '2022-01-01') & (df['Data'] < inicio)]
df_train = df_train.set_index(['Data','acao'])

df_test = df[(df['Data'] > inicio)]
df_test = df_test.set_index(['Data','acao'])

y_train = df_train['resultado_binario']
x_train = df_train.drop(columns=['resultado_binario'])

y_test = df_test['resultado_binario']
x_test = df_test.drop(columns=['resultado_binario'])

In [330]:
df_test['resultado_binario'].sum()/df_test.shape[0]

0.2857142857142857

In [331]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [332]:
#clf = ExtraTreesClassifier()
#clf = LogisticRegression()
clf = lgb.LGBMClassifier()
clf.fit(x_train, y_train)

# predict the results
y_pred=clf.predict(x_test)
y_proba = clf.predict_proba(x_test)[:,1]
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

print('Training set score: {:.4f}'.format(clf.score(x_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(x_test, y_test)))

LightGBM Model accuracy score: 0.4857
Training set score: 1.0000
Test set score: 0.4857


In [333]:
y_pred.sum()

12.0

In [334]:
x_test = x_test.reset_index()
x_test['predito'] = y_pred
x_test['resultado_binario'] = y_test.values
x_test['proba'] = y_proba

In [335]:
new = x_test[['Data','acao','resultado_binario','predito']]
new = x_test[(x_test['predito'] == 1) & ((x_test['proba'] >= 0.5))]
new['certo'] = new.apply(lambda x: 1 if x['resultado_binario'] == x['predito'] else 0, axis = 1)

In [336]:
#new[['Data','acao','resultado_binario','predito','certo']]

In [337]:
new['certo'].sum()/new.shape[0]

0.16666666666666666

In [310]:
new.shape

(7, 41)

In [285]:
new

,Data,acao,Open,High,Low,Close,Vol,BB_up,BB_mid,BB_low,...,dist_bb,dist_media,dist_media_abs,dist_price_media,dist_RSI_70,dist_RSI_30,predito,resultado_binario,proba,certo
3,2022-11-10 09:00:00,call,114155.0,114155.0,112410.0,112515.0,323127,123005.661083,114913.50,106821.338917,...,16184.322167,2357.952381,2357.952381,2339.047619,-23.090905,16.909095,1.0,0.0,0.518148,0
5,2022-11-18 09:00:00,sell,111410.0,111935.0,110910.0,111905.0,214351,123030.275027,115108.75,107187.224973,...,15843.050054,347.285714,347.285714,3010.714286,-23.704912,16.295088,1.0,1.0,0.518148,1
6,2022-11-18 13:45:00,call,110355.0,110425.0,109895.0,109920.0,225608,123087.180450,115067.75,107048.319550,...,16038.860901,-1326.666667,1326.666667,4941.666667,-26.311967,13.688033,1.0,1.0,0.518148,1
8,2022-11-24 14:30:00,sell,112415.0,112905.0,112385.0,112885.0,224030,123075.726873,115135.75,107195.773127,...,15879.953747,-3044.380952,3044.380952,1982.380952,-21.167115,18.832885,1.0,1.0,0.577267,1
9,2022-11-28 17:40:00,sell,109435.0,109625.0,109430.0,109615.0,83460,122885.390016,115219.25,107553.109984,...,15332.280033,-4183.857143,4183.857143,5257.857143,-25.642790,14.357210,1.0,1.0,0.806012,1
10,2022-11-29 11:15:00,sell,110445.0,110910.0,110400.0,110885.0,314859,122719.187768,115311.00,107902.812232,...,14816.375535,-4527.857143,4527.857143,4127.857143,-23.588515,16.411485,1.0,0.0,0.607649,0
11,2022-11-30 09:00:00,sell,112000.0,112480.0,111840.0,112470.0,193407,122586.872050,115104.25,107621.627950,...,14965.244101,-4180.714286,4180.714286,2705.714286,-21.055141,18.944859,1.0,1.0,0.517466,1
12,2022-12-05 09:05:00,call,112415.0,112435.0,111830.0,111875.0,261939,122560.866019,114946.75,107332.633981,...,15228.232038,-3404.476190,3404.476190,3075.476190,-21.973019,18.026981,1.0,0.0,0.795347,0
21,2022-12-15 09:00:00,call,105030.0,105310.0,104770.0,104780.0,193777,116599.147711,110151.50,103703.852289,...,12895.295421,-4493.904762,4493.904762,5771.904762,-32.760811,7.239189,1.0,1.0,0.564981,1
22,2022-12-15 10:35:00,sell,105685.0,106125.0,105585.0,106105.0,291171,115116.884515,109554.25,103991.615485,...,11125.269030,-4117.809524,4117.809524,3853.809524,-29.088671,10.911329,1.0,0.0,0.558046,0
